# Bibliotheken

In [1]:
import pandas as pd
import json
import networkx as nx
import re
from sklearn.metrics.pairwise import cosine_similarity

# Daten einlesen

In [7]:
# Plotti hat die Namen und Daten separat in ein File kopiert um daraus den head zu machen.

namen = pd.read_excel("namen.xlsx")
namen.index = namen["CouncillorId"]
namen = namen.T
namen = namen[1:]
namen.head()

CouncillorId,CouncillorBioId,CouncillorName,Rat,Fraktion,Kanton,Geburtsdatum,Vereidigungsdatum,nan
3055,4154,Addor Jean-Luc,NR,V,VS,1964-04-22,2015-11-30,4154
2670,3867,Aebi Andreas,NR,V,BE,1958-11-26,2015-11-30,3867
2760,4049,Aebischer Matthias,NR,S,BE,1967-10-18,2015-11-30,4049
2758,4053,Aeschi Thomas,NR,V,ZG,1979-01-13,2015-11-30,4053
2796,4090,Amaudruz Céline,NR,V,GE,1979-03-15,2015-11-30,4090


In [12]:
# jetzt Daten einlesen - und letzte Spalten weglassen, weil für uns nicht aussagekräftig

df = pd.read_excel("parlament_export_session_5019_de_CH.xlsx")
df = df.drop(['Decision','Ja', "Nein", "Enth.", "Entschuldigung gem. Art. 57 Abs. 4", "Hat nicht teilgenommen", "Präsident"], axis=1)
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
0,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
1,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
2,2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
3,2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
4,2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Index Setzen

In [4]:
df.index = df["AffairTitle"]
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
AffairTitle,,,,,,,,,,,,,,,,,,,,,
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Tabelle 90 Grad Kippen

In [6]:
# Tabelle transponieren, so dass die Geschäfte die Spaltentitel werden mit .T

df_t = df.T
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
VoteDate,2019-09-09 15:26:55,2019-09-09 15:27:54,2019-09-09 16:16:12,2019-09-09 17:33:42,2019-09-09 17:34:46,2019-09-09 17:36:08,2019-09-09 17:37:27,2019-09-09 17:38:25,2019-09-09 17:39:22,2019-09-09 17:40:54,...,2019-09-27 09:23:53,2019-09-27 09:24:41,2019-09-27 09:44:37,2019-09-27 09:45:36,2019-09-27 09:46:26,2019-09-27 09:47:14,2019-09-27 09:48:23,2019-09-27 09:49:33,2019-09-27 09:50:25,2019-09-27 09:54:11
Rat,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR,...,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR
Kommission,"UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR",...,"RK-NR, WBK-SR","RK-NR, RK-SR","WAK-NR, WAK-SR","WBK-NR, SGK-SR","SGK-NR, SGK-SR","UREK-NR, UREK-SR","WBK-NR, SGK-SR","APK-NR, APK-SR","SiK-NR, SiK-SR","SPK-NR, SPK-SR"
Dept.,UVEK,UVEK,Parl,Parl,Parl,Parl,Parl,Parl,Parl,Parl,...,EJPD,EJPD,EFD,EDI,EDI,UVEK,EDI,EJPD,VBS,WBF
AffairId,18.097,18.097,17.405,17.405,17.405,17.405,17.405,17.405,17.405,17.405,...,17.069,18.049,18.05,18.052,18.091,18.095,18.441,19.017,19.022,19.024


# Ersten Infofelder übergehen

In [19]:
# Wir wollen alles ab der 12. Zeile. Das davor interessiert uns nicht.

df_t = df_t[12:]
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4215,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1
4025,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,3,1,0
4156,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4157,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4101,1,1,1,0,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Kategorien durch Zahlen ersetzen

In [20]:
# Damit wir das ganze verarbeitbar haben, ersetzten wir die Werte ja nein, etc mit Zahlen.

cleanup_dict = {"Ja": 1, "Nein": 0, "Hat nicht teilgenommen": 2, "Enthaltung": 3, "Der Präsident stimmt nicht": 4, "Entschuldigt": 5}

In [24]:
#df_t.replace(cleanup_dict, inplace=True)
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4215,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1
4025,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,3,1,0
4156,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4157,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4101,1,1,1,0,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Cosine Similarity berechnen

In [25]:
# Cosine similarity vergleicht die Verhältnisse zwischen den Werten einer Matrix

    leute
    [[1,0,1,0],
     [0,0,0,0],
     [1,0,1,1]]
Leute     
Unter Berücksichtigung aller Abstimmungen


In [13]:


pd.DataFrame(cosine_similarity([[1,0,1,0],[0,0,0,0],[1,0,1,1]]))

,0,1,2
0,1.000000,0.0,0.816497
1,0.000000,0.0,0.000000
2,0.816497,0.0,1.000000


In [28]:
# Cosine similarity für unser df berechnen

dists = cosine_similarity(df_t)
dists_named = pd.DataFrame(dists, columns=df_t.index)
dists_named.index = df_t.index
# Weil es zu viele Kanten wären wenn wir alle berücksichtigen würden, schmeissen wir solche unter 0.8 raus
filtered = dists_named[dists_named > 0.8]
filtered = filtered.fillna(0)
filtered.head()

,4215,4025,4156,4157,4101,4158,4189,4200,4070,3830,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
4215,1.000000,0.000000,0.000000,0.000000,0.807307,0.000000,0.848068,0.816673,0.00000,0.809568,...,0.855717,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4025,0.000000,1.000000,0.800386,0.000000,0.000000,0.821256,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.836551,0.809543
4156,0.000000,0.800386,1.000000,0.833544,0.000000,0.928008,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.878534,0.0,0.945142,0.851254
4157,0.000000,0.000000,0.833544,1.000000,0.000000,0.830712,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.846499,0.844855
4101,0.807307,0.000000,0.000000,0.000000,1.000000,0.000000,0.811211,0.845894,0.93995,0.847244,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000


# Als Netzwerk exportieren

In [29]:
# Wir machen aus Namen und Wert einen Dict

namen_dict = dict(zip(namen["CouncillorBioId"],namen["CouncillorName"]))
fract_dict = dict(zip(namen["CouncillorBioId"],namen["Fraktion"]))
name_mappings = {}
fract_mappings = {}
i = 0
for id in df_t.index:
    name_mappings[i] = namen_dict[id]
    fract_mappings[i] = fract_dict[id]
    i += 1

In [30]:
G=nx.from_numpy_matrix(filtered.values)
nx.set_node_attributes(G, values = fract_mappings, name='fraktion')
H = nx.relabel_nodes(G, name_mappings)
nx.write_gexf(H,"Politics.gexf")